In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import os 
from model.SimCLR import SimCLR
from model.Train import Trainer
from model.Losses import NTXentLoss
from utils.DataLoaderDegradation import DataLoaderDegradation as DLD
from torch.utils.data import DataLoader, random_split

In [ ]:
torch.cuda.empty_cache()

In [4]:
SSH = os.getcwd() == 'c:\\Cours-Sorbonne\\M1\\Stage\\src'

True

In [ ]:
path_rol_comp = "C:/Cours-Sorbonne/M1/Stage/src/data/rol_compressed" 
path_rol_super_comp = "C:/Cours-Sorbonne/M1/Stage/src/data/rol_super_compressed" 
path_sim_rol_extracted_comp = "C:/Cours-Sorbonne/M1/Stage/src/data/similaires_rol_extracted_nn_compressed" 
path_filtered = "C:/Cours-Sorbonne/M1/Stage/src/data/rol_compressed/json_filtered"
path_targets = "C:/Cours-Sorbonne/M1/Stage/src/data/rol_sim_rol_triplets/targets.npy"

if SSH:
    path_rol_comp = "../data/rol_super_compressed" 
    path_sim_rol_extracted_comp = "../data/sim_rol_super_compressed" 
    path_filtered = "../data/rol_super_compressed/json_filtered"
    path_targets = "../data/rol_sim_rol_couples/targets.npy"

## Params

In [ ]:
epochs = 20
image_size = 128
num_classes = 1
batch_size = 8
learning_rate = 1e-4
train_ratio = 0.8
val_ratio = 0.2
temperature = 0.5

In [ ]:
dataset = DLD(path_rol_comp, path_sim_rol_extracted_comp, path_filtered, shape=(256, 256), target_path=path_targets)

train_size = int(train_ratio * len(dataset))
val_size = len(dataset) - train_size

train_set, val_set = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=True)

In [ ]:
DLD.show_data(train_loader)
DLD.show_data(val_loader)

# Approche Supervisé

In [ ]:
model = SimCLR(feature_size=128)
optimizer = torch.optim.AdamW
loss_fn = NTXentLoss(temperature=temperature)

In [ ]:
trainer = Trainer()
trainer.set_model(model, "SimCLR-128") \
.set_optimizer(optimizer) \
.set_loss(loss_fn) 

model = trainer.fit(train_data=train_loader, validation_data=val_loader, learning_rate=learning_rate, verbose=True, epochs=epochs, sim_clr=True)

In [ ]:
trainer.save("model_simclr.pth","history_simclr.txt")

# Approche Non Supervisé